In [3]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [18]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

In [19]:

# lee mobiliarios
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")

# Lee equipos medicos y los ordena
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")
df_equipos_medicos = df_equipos_medicos.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")
df_industriales = df_industriales.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [31]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)

# Deja solamente las columnas utiles
columnas_utiles = [
    "correlativo_antiguo",
    "n_inventario_2025",
    "bien",
    "marca",
    "modelo",
    "serie",
    "tipo_bien",
    "piso",
    "unidadservicio_clinico",
    "ubicacion_unidad",
    "propiedad",
    "observaciones",
]

# Agrega el conteo de bienes
union_bienes_con_informatico["conteo"] = 1

conteo_equipos = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

In [32]:
union_bienes_con_informatico

,correlativo_antiguo,n_inventario_2025,bien,marca,modelo,serie,tipo_bien,piso,unidadservicio_clinico,ubicacion_unidad,propiedad,observaciones,unidad_hoja,correlativo_2025,tipo,oc,n_factura,ingreso,conteo
0,1499-6132,2025-01,ESTACION DE TRABAJO,NaN,NaN,NaN,MOBILIARIO,1.0,CONGENITOS,EMO,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1499-1685,2025-02,SILLA ESCRITORIO,NaN,NaN,NaN,MOBILIARIO,1.0,CONGENITOS,EMO,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1499-1633,2025-03,SILLA ESCRITORIO,NaN,NaN,NaN,MOBILIARIO,1.0,CONGENITOS,EMO,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1499-6135,2025-04,LIBRERO,NaN,NaN,NaN,MOBILIARIO,1.0,CONGENITOS,EMO,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1499-1692,2025-05,SILLA ESCRITORIO,NaN,NaN,NaN,MOBILIARIO,1.0,CONGENITOS,EMO,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,sin código,NaN,ESCANER PORTATIL,EPSON,J291A,Q5EZ789408,INFORMATICO,3,MQ 3 SUR,DENISSE GALLARDO,DONACION,NaN,NaN,NaN,ACCESORIO,NaN,NaN,NaN,1
553,sin código,NaN,ESCANER PORTATIL,BROTHERS,DSMOBILE 620,U63543E3U114222,INFORMATICO,2,BRONCOSCOPIA,MELISSA MORALES ENFERMERA,INT,NaN,NaN,NaN,ACCESORIO,NaN,NaN,NaN,1
554,sin código,NaN,ESCANER PORTATIL,BROTHERS,DSMOBILE 620,U63543E3U112700,INFORMATICO,4,ECOCARDIOGRAMA,SALA DE ECOCARDIOGRAMA,INT,NaN,NaN,NaN,ACCESORIO,NaN,NaN,NaN,1
555,sin código,NaN,ESCANER PORTATIL,BROTHERS,DSMOBILE 620,U63543E3U112911,INFORMATICO,2,IMAGENOLOGIA,SALA INFORMES,INT,NaN,NaN,NaN,ACCESORIO,NaN,NaN,NaN,1


## Asignacion de Correlativos

Se hara de forma separada

In [8]:
GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO = ["INT", "INT (SSMO)", "INT (MINSAL)", "COMODATO"]
GLOSAS_INDUSTRIALES_CON_CORRELATIVO = ["INT", "INT (MEL)", "INT (DONACION)", "COMODATO"]

In [9]:
ULTIMO_CORRELATIVO = 4015 + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativo_medicos = df_equipos_medicos["propiedad"].isin(
    GLOSAS_EQUIPOS_MEDICOS_CON_CORRELATIVO
)
df_equipos_medicos_con_correlativos = df_equipos_medicos[mask_correlativo_medicos]

# Asigna correlativos a los registros que requieren correlativo
correlativos_equipos_medicos = [
    f"2025-{i}"
    for i in range(
        ULTIMO_CORRELATIVO, ULTIMO_CORRELATIVO + len(df_equipos_medicos_con_correlativos)
    )
]
df_equipos_medicos.loc[mask_correlativo_medicos, "correlativo_2025"] = correlativos_equipos_medicos
df_equipos_medicos["correlativo_2025"] = df_equipos_medicos["correlativo_2025"].replace(
    "nan", np.nan
)

In [10]:
# Aisla el ultimo correlativo asignado
ULTIMO_CORRELATIVO_MEDICOS = int(
    df_equipos_medicos["correlativo_2025"].dropna().iloc[-1].split("-")[-1]
)

PRIMER_CORRELATIVO_INDUSTRIALES = ULTIMO_CORRELATIVO_MEDICOS + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativos_ind = df_industriales["propiedad"].isin(GLOSAS_INDUSTRIALES_CON_CORRELATIVO)
df_industriales_con_correlativos = df_industriales[mask_correlativos_ind]

# Asigna los correlativos a los registros que requieren correlativo
correlativos_industriales = [
    f"2025-{i}"
    for i in range(
        PRIMER_CORRELATIVO_INDUSTRIALES,
        PRIMER_CORRELATIVO_INDUSTRIALES + len(df_industriales_con_correlativos),
    )
]

df_industriales.loc[mask_correlativos_ind, "correlativo_2025"] = correlativos_industriales
df_industriales["correlativo_2025"] = df_industriales["correlativo_2025"].replace("nan", np.nan)

In [11]:
# Genera la matriz total de bienes
matriz_total_correlativos = pd.concat([df_equipos_medicos, df_industriales])
matriz_total_correlativos = matriz_total_correlativos.drop(columns="unidad_hoja")

# Reordena columnas de matriz total
orden_columnas = ["correlativo_antiguo", "correlativo_2025"] + list(
    matriz_total_correlativos.columns.drop(["correlativo_antiguo", "correlativo_2025"])
)
matriz_total_correlativos = matriz_total_correlativos[orden_columnas]

In [12]:
# Exporta asignando correlativos
df_equipos_medicos.to_excel("../data/interim/matriz_equipos_medicos_2025.xlsx", index=False)
df_industriales.to_excel("../data/interim/matriz_industriales_2025.xlsx", index=False)

In [13]:
fa.guardar_dataframe_como_tabla_excel(
    matriz_total_correlativos,
    "../data/interim/matriz_bienes_INT_2025.xlsx",
    nombre_tabla="MatrizBienes",
    estilo_tabla="TableStyleMedium1"
)

✅ Archivo guardado exitosamente en: ../data/interim/matriz_bienes_INT_2025.xlsx


## Exportar bases para etiquetas

In [14]:
# Aisla la matriz de equipos medicos
for unidad in matriz_total_correlativos.query("tipo_bien == 'EQUIPO MEDICO'")[
    "unidadservicio_clinico"
].unique():
    # Aisla la unidad
    df_equipos_medicos_unidad = df_equipos_medicos.query("unidadservicio_clinico == @unidad")

    fa.guardar_dataframe_como_tabla_excel(
        df_equipos_medicos_unidad,
        f"../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS {unidad}.xlsx",
        nombre_tabla="MatrizBienes",
        estilo_tabla="TableStyleMedium1",
    )

✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MEDICINA FISICA Y REHABILITACION.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS CONSULTORIO EXTERNO.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS LABORATORIO.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS UNIDAD DE APOYO FARMACIA.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS FUNCION PULMONAR.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS UMT.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MQ 3 NORTE.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MQ 3 SUR.xl

In [15]:
matriz_total_correlativos.query("tipo_bien == 'EQUIPO MEDICO'")["propiedad"].value_counts()

propiedad
INT                         872
COMODATO                     91
ARRIENDO                      4
PRESTAMO HOSMET               3
INT (SSMO)                    3
INT (MINSAL)                  2
REVISAR DONACION GRIFOLS      1
DRA. LINACRE                  1
Name: count, dtype: int64